In [7]:
import pandas as pd
import numpy as np
from sklearn import metrics
import tensorflow as tf
import binascii
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [2]:
csv_data = pd.read_csv('testing.csv')

In [3]:
csv_data

,length,timestamp,eth.src,eth.dst,eth.type,ip.src,ip.dst,ip.version,ip.proto,ip.len,...,load_13,load_14,load_15,load_16,load_17,load_18,load_19,protocol,protocol.sport,protocol.dport
0,144,1.646950e+09,3e:b1:55:13:9d:6b,4e:78:53:ac:f3:f6,2048,10.244.0.19,10.244.0.18,4,6,130,...,5,4,12,4,5,4,3,TCP,47144,3306
1,144,1.646950e+09,0e:e4:01:d4:1f:8a,4e:78:53:ac:f3:f6,2048,10.244.0.19,10.244.0.18,4,6,130,...,5,4,12,4,5,4,3,TCP,47144,3306
2,66,1.646950e+09,4e:78:53:ac:f3:f6,3e:b1:55:13:9d:6b,2048,10.244.0.18,10.244.0.19,4,6,52,...,0,0,0,0,0,0,0,TCP,3306,47144
3,78,1.646950e+09,4e:78:53:ac:f3:f6,3e:b1:55:13:9d:6b,2048,10.244.0.18,10.244.0.19,4,6,64,...,0,0,0,0,0,0,0,TCP,3306,47144
4,197,1.646950e+09,4e:78:53:ac:f3:f6,3e:b1:55:13:9d:6b,2048,10.244.0.18,10.244.0.19,4,6,183,...,0,0,0,0,2,0,3,TCP,3306,47144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76014,374,1.646950e+09,22:4a:1a:68:48:92,0e:e4:01:d4:1f:8a,2048,10.244.0.14,20.126.255.241,4,6,360,...,4,15,4,2,15,6,13,TCP,44008,9000
76015,66,1.646950e+09,0e:e4:01:d4:1f:8a,22:4a:1a:68:48:92,2048,20.126.255.241,10.244.0.14,4,6,52,...,0,0,0,0,0,0,0,TCP,9000,44008
76016,198,1.646950e+09,0e:e4:01:d4:1f:8a,22:4a:1a:68:48:92,2048,20.126.255.241,10.244.0.14,4,6,184,...,8,7,4,1,0,3,7,TCP,9000,44008
76017,66,1.646950e+09,22:4a:1a:68:48:92,0e:e4:01:d4:1f:8a,2048,10.244.0.14,20.126.255.241,4,6,52,...,0,0,0,0,0,0,0,TCP,44008,9000


In [4]:
csv_data.columns

Index(['length', 'timestamp', 'eth.src', 'eth.dst', 'eth.type', 'ip.src',
       'ip.dst', 'ip.version', 'ip.proto', 'ip.len', 'ip.ihl', 'ip.tos',
       'ip.ttl', 'load.count', 'FIN', 'SYN', 'RST', 'PSH', 'ACK', 'URG', 'ECE',
       'CWR', 'UNK', 'source_docker_bridge', 'source_dns', 'source_service',
       'source_pod', 'source_external', 'destination_docker_bridge',
       'destination_dns', 'destination_service', 'destination_pod',
       'destination_external', 'load_0', 'load_1', 'load_2', 'load_3',
       'load_4', 'load_5', 'load_6', 'load_7', 'load_8', 'load_9', 'load_10',
       'load_11', 'load_12', 'load_13', 'load_14', 'load_15', 'load_16',
       'load_17', 'load_18', 'load_19', 'protocol', 'protocol.sport',
       'protocol.dport'],
      dtype='object')

In [8]:
import custom_preprocessing
final_df = custom_preprocessing.process_df(csv_data)

In [9]:
csv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76019 entries, 0 to 76018
Data columns (total 56 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   length                     76019 non-null  int64  
 1   timestamp                  76019 non-null  float64
 2   eth.src                    76019 non-null  object 
 3   eth.dst                    76019 non-null  object 
 4   eth.type                   76019 non-null  int64  
 5   ip.src                     76019 non-null  object 
 6   ip.dst                     76019 non-null  object 
 7   ip.version                 76019 non-null  int64  
 8   ip.proto                   76019 non-null  int64  
 9   ip.len                     76019 non-null  int64  
 10  ip.ihl                     76019 non-null  int64  
 11  ip.tos                     76019 non-null  int64  
 12  ip.ttl                     76019 non-null  int64  
 13  load.count                 76019 non-null  int

In [14]:
list_of_cols_straight = ['length', 'eth.type', 'ip.version', 'ip.proto', 'ip.len', 'ip.ihl', 'ip.tos',
       'ip.ttl' ,'load.count', 'FIN', 'SYN', 'RST', 'PSH', 'ACK',
       'URG', 'ECE', 'CWR', 'UNK', 'source_docker_bridge', 'source_dns',
       'source_service', 'source_pod', 'source_external',
       'destination_docker_bridge', 'destination_dns', 'destination_service',
       'destination_pod', 'destination_external', 'load_0', 'load_1', 'load_2',
       'load_3', 'load_4', 'load_5', 'load_6', 'load_7', 'load_8', 'load_9',
       'load_10', 'load_11', 'load_12', 'load_13', 'load_14', 'load_15',
       'load_16', 'load_17', 'load_18', 'load_19',
       'protocol.sport', 'protocol.dport']

for col in list_of_cols_straight:
    final_df[str(col)] = list(csv_data[str(col)]) 

In [15]:
dummy_protocol = pd.get_dummies(csv_data['protocol'])
final_df['TCP'] = list(dummy_protocol['TCP'])
final_df['UDP'] = list(dummy_protocol['UDP'])

In [22]:
final_df = final_df.drop(['protocol'], axis = 1)
# 

# md = csv_data['load'][0]

# final_df[[f'load_{x}' for x in range(20)]] = None

In [23]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76019 entries, 0 to 76018
Data columns (total 72 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   octet_1_eth_dst            76019 non-null  int64 
 1   octet_2_eth_dst            76019 non-null  int64 
 2   octet_3_eth_dst            76019 non-null  int64 
 3   octet_4_eth_dst            76019 non-null  int64 
 4   octet_5_eth_dst            76019 non-null  int64 
 5   octet_6_eth_dst            76019 non-null  int64 
 6   octet_1_eth_src            76019 non-null  int64 
 7   octet_2_eth_src            76019 non-null  int64 
 8   octet_3_eth_src            76019 non-null  int64 
 9   octet_4_eth_src            76019 non-null  int64 
 10  octet_5_eth_src            76019 non-null  int64 
 11  octet_6_eth_src            76019 non-null  int64 
 12  octet_1_ip_src             76019 non-null  object
 13  octet_2_ip_src             76019 non-null  object
 14  octet_

In [ ]:
# hex_lambda = lambda x: int(x,16)

# clean_df = pd.DataFrame()

# clean_df[[f'load_{x}' for x in range(20)]] = None

# for index, row in csv_data['load'].iteritems():
#     if row != str(0):
#         if len(row)!=20:
#             row = row + '0'*(20-len(row))
#         clean_df.loc[index] = [hex_lambda(ch) for ch in row]
#     else:
#         clean_df.loc[index] = [0]*20

In [ ]:
# new_df = pd.concat([final_df, clean_df], axis = 1)

In [ ]:
# csv_data.columns

In [ ]:
new_df

In [ ]:
new_df.columns

In [27]:
from sklearn.preprocessing import StandardScaler
columns_not_to_scale = ['FIN', 'SYN', 'RST', 'PSH', 'ACK', 'URG', 'ECE', 'CWR', 'UNK',
       'source_docker_bridge', 'source_dns', 'source_service', 'source_pod',
       'source_external', 'destination_docker_bridge', 'destination_dns',
       'destination_service', 'destination_pod', 'destination_external', 'TCP',
       'UDP']

for col in final_df.columns:
    if col not in columns_not_to_scale:
        scale = StandardScaler().fit(final_df[[col]])
        final_df[col] = scale.transform(final_df[[col]])

In [28]:
final_df

,octet_1_eth_dst,octet_2_eth_dst,octet_3_eth_dst,octet_4_eth_dst,octet_5_eth_dst,octet_6_eth_dst,octet_1_eth_src,octet_2_eth_src,octet_3_eth_src,octet_4_eth_src,...,load_14,load_15,load_16,load_17,load_18,load_19,protocol.sport,protocol.dport,TCP,UDP
0,0.150279,-0.707455,0.189970,0.559643,1.266455,1.433830,0.039796,0.172766,0.450797,-1.928221,...,0.847614,2.146802,0.789267,0.928004,0.836737,0.413879,0.858379,-0.955285,1,0
1,0.150279,-0.707455,0.189970,0.559643,1.266455,1.433830,-0.868310,1.072913,-1.067054,0.885951,...,0.847614,2.146802,0.789267,0.928004,0.836737,0.413879,0.858379,-0.955285,1,0
2,-0.167482,0.409109,0.230950,-1.495849,0.122689,-0.886049,0.342498,-0.833281,0.414658,0.302703,...,-0.483793,-0.535350,-0.495837,-0.526869,-0.489948,-0.509764,-1.106177,1.022932,1,0
3,-0.167482,0.409109,0.230950,-1.495849,0.122689,-0.886049,0.342498,-0.833281,0.414658,0.302703,...,-0.483793,-0.535350,-0.495837,-0.526869,-0.489948,-0.509764,-1.106177,1.022932,1,0
4,-0.167482,0.409109,0.230950,-1.495849,0.122689,-0.886049,0.342498,-0.833281,0.414658,0.302703,...,-0.483793,-0.535350,-0.495837,0.055080,-0.489948,0.413879,-1.106177,1.022932,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76014,-1.120762,1.408140,-1.490181,1.097027,-1.553061,-0.368666,-0.489932,-1.645179,-0.615312,-0.688819,...,4.508983,0.358701,0.146715,3.837752,1.500080,3.492692,0.717842,-0.698339,1,0
76015,-0.723562,-1.608541,-0.977940,-0.353909,-1.007777,-0.235148,-0.868310,1.072913,-1.067054,0.885951,...,-0.483793,-0.535350,-0.495837,-0.526869,-0.489948,-0.509764,-0.851006,0.881418,1,0
76016,-0.723562,-1.608541,-0.977940,-0.353909,-1.007777,-0.235148,-0.868310,1.072913,-1.067054,0.885951,...,1.846169,0.358701,-0.174561,-0.526869,0.505066,1.645404,-0.851006,0.881418,1,0
76017,-1.120762,1.408140,-1.490181,1.097027,-1.553061,-0.368666,-0.489932,-1.645179,-0.615312,-0.688819,...,-0.483793,-0.535350,-0.495837,-0.526869,-0.489948,-0.509764,0.717842,-0.698339,1,0


In [29]:
final_df = final_df.astype('float64')

In [31]:
from sklearn.model_selection import train_test_split

x_normal = final_df.values

x_normal_train, x_normal_test = train_test_split(
    x_normal, test_size=0.25, random_state=42)

In [32]:
x_normal.shape

(76019, 72)

In [ ]:
# #training on the normal data
# #autoencoder 
# model = Sequential()
# model.add(Dense(40, input_dim=x_normal.shape[1], activation='relu'))
# model.add(Dense(5, activation='relu')) # size to compress to
# model.add(Dense(40, activation='relu'))
# model.add(Dense(x_normal.shape[1])) # Multiple output neurons
# model.compile(loss='mean_squared_error', optimizer='adam')
# model.fit(x_normal_train,x_normal_train,verbose=1,epochs=120)